## Usage
This notebook takes a file containing complex information and  a mask file, creates the phase, and saves the results in `outpath`. 

### Parameters
`inpath` : str    
    The full path to the file containing complex information.

`maskpath` : str    
    The full path to the file with the mask.

`normalise` : bool    
    Set as True to apply normalisation.

`outpath` : str    
    The full path to the output folder. 


**The parameters should be provided by explicitly modifying the top cell content or using tools such as [papermill](https://papermill.readthedocs.io/en/latest/index.html). If the notebook is run as is, please define the parameters accordingly.**

### Dependencies
- numpy
- ptypy

In [ ]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# import ptypy.utils as u

In [ ]:
with h5py.File(inpath, "r") as f:
    data = f["entry/data/data"][:]

In [ ]:
if maskpath:
    try:
        with h5py.File(maskpath, "r") as f:
            background_mask = f["entry/data/background_mask"][:]
            sample_mask = f["entry/data/sample_mask"][:]
    except FileNotFoundError:
        print(f"Mask file {maskpath} is not found.")


In [ ]:
if not maskpath:
    sample_mask = np.ones(np.shape(data), dtype=np.bool)    
    background_mask = np.zeros(np.shape(data), dtype=np.bool)

## Initialise np array

In [ ]:
phase_final = np.empty(np.shape(data), dtype=np.float32)

## Phase calculation

In [ ]:
from numpy._typing import NDArray

def get_phase(data: NDArray, background_mask: NDArray, 
              sample_mask: NDArray, normalise: bool) -> NDArray:
    """Return phase

    Parameters
    ----------
    data : NDArray
        The loaded complex data.
    background_mask : NDArray
        Mask, boolean values.
    sample_mask : NDArray
        Mask, boolean values of sample alone.
    normalise : bool
        True if obj_phase_bg_at_zero should be used

    Returns
    -------
    NDArray
    """
    # bgvalue = np.median(np.angle(data[background_mask]))
    # obj_ramp_removed = u.rmphaseramp(data, weight=background_mask) 
    # obj_phase_bg_at_zero = obj_ramp_removed * np.exp(-1j * bgvalue)
    # phase = np.angle(obj_phase_bg_at_zero) if normalise else (np.angle(obj_ramp_removed) - bgvalue)
    # phase_norm = normalise_phase(phase, background_mask)

    phase_norm = np.angle(data)
    return phase_norm


def normalise_phase(phase_in: NDArray, background_mask: NDArray) -> NDArray:
    """Return phase

    Parameters
    ----------
    phase : NDArray
        The loaded phase data.
    background_mask : NDArray
        Mask of boolean values. 

    Returns
    -------
    NDArray
    """
    med = np.zeros((len(phase_in)))
    phase_normalised = np.copy(phase_in)
    for i in range(phase_in.shape[0]):
        med[i] = np.median(phase_in[i][background_mask[i]])
        phase_normalised[i] = phase_in[i] - med[i]
    return phase_normalised

## Process image stack

In [ ]:
for i in range(len(data)):
    phase_final[i] = get_phase(data[i], background_mask[i], sample_mask[i], normalise)

## Copy the original file and save to output filepath

In [ ]:
# If an output file isn't specified, save the output file to the same directory as the input data  
outpath = Path(inpath).parent if not outpath else Path(outpath)

In [ ]:
# Follow the naming convention of the input file
outfile_stem = Path(inpath).stem
outfile = outpath.joinpath(f"{outfile_stem}_phase.nxs")

In [ ]:
os.system(f"cp {inpath} {outfile}")
with h5py.File(outfile, "r+") as f:
    del f["entry/data/data"]
    f[f"entry/data/phase"] = phase_final
    f[f"entry/data/background_mask"] = background_mask    
    f[f"entry/data/sample_mask"] = sample_mask

## Figures

In [ ]:
start = 0
mid = np.rint(phase_final.shape[0]/2).astype(int)
end = phase_final.shape[0]-1

fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True)
ax1.set_title("First slice")
ax1.imshow(phase_final[start])
ax2.set_title("Mid slice")
ax2.imshow(phase_final[mid])
ax3.set_title("End slice")
ax3.imshow(phase_final[end])

plt.suptitle('Phase', fontsize=12)
plt.subplots_adjust(top=1.4)
plt.show()